In [115]:
## 단위 EDI 엑셀 파일을 읽어 들여서 dt_trial_balance 파일의 내용을 넣는다

In [39]:
import pandas as pnds
import openpyxl
import datetime
import pickle

# 1. EDI 템플릿 엑셀 파일을 읽어서 DataFrame 생성
excel_filename = './EDI-xx월.xlsx'                                  # EDI 엑셀파일
edi_excel = openpyxl.load_workbook(excel_filename, data_only=False)  # 수식파일 포함하여 엑셀파일 읽어 들임
ws = edi_excel.active                                               # 활성화된 sheet

yesterday = datetime.datetime.now() - datetime.timedelta(1) # 어제 날짜 추출
target_date = yesterday.strftime('%Y%m%d')                  # 어제 날짜 포맷
opera_date = yesterday.strftime('%b.%d.%Y')                 # 어제 날짜 오페라 포맷

excel_filename = './EDI-' + str(yesterday.day) + '월.xlsx'   # 파일 저장 시에 필요한 당 월의 엑셀파일 이름

# 엑셀 화면 출력
# for row in ws.iter_rows(min_row=1):
#     print([col.value for col in row])
for row in ws:
    print([cell.value for cell in row])


1
2


AttributeError: 'MergedCell' object has no attribute 'col_idx'

In [ ]:
# 2. 오페라 어제 마감된 dataframe trial report(dt_trial_balance_날짜) 읽기
# target_date = (datetime.datetime.now() - datetime.timedelta(1)).strftime("%Y%m%d")      # 어제 날짜 추출
target_date = '20220713'
base_dir = './data/' + target_date + '/dfdata/'
data_dir = base_dir + 'df_opera_trial_' + target_date
with open(data_dir, "rb") as file:
    opera_df = pickle.load(file)
print(opera_df)
# opera_df['TRX_CODE'] = opera_df['TRX_CODE'].astype('str', errors='ignore')      # 일단, str타입으로 변경

                  DESCRIPTION  TB_AMOUNT  NON_REVENUE_AMT    TRX_DATE
TRX_CODE                                                             
9233       Manual Master Card    -318421           318421  2022-07-13
9237           Manual BC Card    -214330           214330  2022-07-13
9238           Manual KM Card    -384899           384899  2022-07-13
9239      Manual Samsung Card     -88825            88825  2022-07-13
9240           Manual NH Card          0                0  2022-07-13
9242           Manual SH Card    -153560           153560  2022-07-13
9243          Manual KEB Card    -155166           155166  2022-07-13
9245           Manual HD Card     -11000            11000  2022-07-13


In [ ]:
# 3. 오페라 값이 EDI엑셀에 입력될 정보
# # 3-1. 오페라 신용카드 transaction code 추출 : 실행파일 opera_card_trx_code.py(ipynb)에서 생성
# card_df = pnds.read_csv('./trx_card_codes.csv', sep='\t')   # {'TRX_CODE': 'Description'} 형식
# card_df.set_index(keys=['TRX_CODE'], inplace=True)          # transaction code를 index로 변경
# print(card_df)

map_code_edi = {                        # 오페라 dataframe의 trx_code와 EDI 엑셀 좌표 매칭 정보
    # TRX_CODE : 엑셀좌표(A1, B13..)
    '9231': 'B7',
    '9232': 'B9',
    '9233': 'B8',
    '9234': 'B15',
    '9235': 'B6',
    '9236': 'B3',
    '9237': 'B12',
    '9238': 'B14',
    '9239': 'B10',
    '9240': 'B16',
    '9241': 'B17',
    '9242': 'B11',
    '9243': 'B5',
    '9244': 'B4',
    '9245': 'B15'
}

# trial balance 내용을 순서대로 돌면서 엑셀에 입력
for code in opera_df.index:     # trx-code looping
    ws[map_code_edi[code]] = opera_df.loc[code, 'TB_AMOUNT'] * -1       # -1을 곱한 값
ws['B1'] = opera_date           # 날짜 셋팅

# 파일 저장
# edi_excel.save(excel_filename)
# 엑셀 화면 출력
for row in ws.iter_rows(min_row=1):
    print([col.value for col in row])

['Date', 'Jul.20.2022', None, None, 'Total', None, None, 'Collected', 'Collected', 'Balance', 'Repurchase', 'Repurchase', 'Collection', 'Net']
[None, 'BOOK.BL', 'Actual ', 'DIFF', 'BOOK.BL', 'Actual ', 'DIFF', 'Amount', 'Date', None, None, 'Date', None, 'Balance']
['L.A', None, None, '=B3-C3', '=B3', '=C3', '=E3-F3', None, None, '=F3-H3', None, None, '=J3', '=J3-M3']
['LT', None, None, '=B4-C4', '=B4', '=C4', '=E4-F4', None, None, '=F4-H4', None, None, '=J4', '=J4-M4']
['KEB', 155166, None, '=B5-C5', '=B5', '=C5', '=E5-F5', None, None, '=F5-H5', '`', None, '=J5', '=J5-M5']
['JCB', None, None, '=B6-C6', '=B6', '=C6', '=E6-F6', None, None, '=F6-H6', None, None, '=J6', '=J6-M6']
['VISA', None, None, '=B7-C7', '=B7', '=C7', '=E7-F7', None, None, '=F7-H7', None, None, '=J7', '=J7-M7']
['MASTER', 318421, None, '=B8-C8', '=B8', '=C8', '=E8-F8', None, None, '=F8-H8', None, None, '=J8', '=J8-M8']
['S.A', None, None, '=B9-C9', '=B9', '=C9', '=E9-F9', None, None, '=F9-H9', None, None, '=J9', '=J9

In [ ]:
# 내일 날짜는 제외
# date_str = (work_date + datetime.timedelta(1)).strftime("%Y-%m-%d")                                     # 내일 날짜 포맷
# card_history_df = card_history_df[card_history_df['date'].str.contains(date_str, na=False) == False]

# tomorrow = (work_date + datetime.timedelta(1)).strftime('%d')                                                   # 내일 날짜의 day: 'DD' (두자리 문자열)
# card_history_df = card_history_df[card_history_df['거래고유번호'].str.startswith(tomorrow, na=False) == False]


In [ ]:
# # 10-4. 기존의 KICC 카드거래내역 엑셀 파일에 추가 모드로 저장
# try:
#     df_filename = 'df_kicc_history_' + target_date + '.xlsx'            # 저장파일 'df_kicc_history_YYYYMMDD.xlsx
#     print(target_dir + df_filename)

#     with pnds.ExcelWriter(target_dir + df_filename, mode='a', engine='openpyxl') as writer:     # 기존 파일 추가 mode 셋팅
#         grouped_df.to_excel(writer, sheet_name='groupped_calc', index=True)
# except Exception as e:
#     with open('./error.log', 'a') as file:
#         file.write(
#             f'[edi_opera.py - Making Excel] <{datetime.datetime.now()}> Pandas.ExcelWriter error ({df_filename}) ===> {e}\n'
#         )
#     raise(e)